In [72]:
%load_ext autoreload
%autoreload 2

import sys, os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12
from prompters import padding
from utils.parser_utils import get_args

import easydict

import torch
import torch.nn as nn
import numpy as np

import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim

from loss import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. u, s, v = torch.svd(param, some=False)를 수행할 때, 
## s는 행렬이 아닌 값의 형태로 나온다. 그러므로 torch.diag로 행렬화를 시켜야한다

In [73]:
random_matrix = torch.rand(3, 3)
print("원본 행렬:")
print(random_matrix)

# 특이값 분해 수행
u, s, v = torch.svd(random_matrix)

print("U 행렬:")
print(u)
print("특이값(Singular values):")
print(s)
print("V 전치 행렬:")
print(v)

원본 행렬:
tensor([[0.0116, 0.5807, 0.2184],
        [0.0491, 0.7608, 0.3512],
        [0.2313, 0.3778, 0.8771]])
U 행렬:
tensor([[-0.4285, -0.4581, -0.7788],
        [-0.5982, -0.5022,  0.6245],
        [-0.6772,  0.7335, -0.0588]])
특이값(Singular values):
tensor([1.3279, 0.5401, 0.0084])
V 전치 행렬:
tensor([[-0.1438,  0.2587,  0.9552],
        [-0.7228, -0.6868,  0.0772],
        [-0.6760,  0.6793, -0.2858]])


### SVD(Singular Value Decomposition)는 정확한 복원이 아닌 근사적인 복원이다.. 
### SVD는 정보 손실이 발생할 수 있는 근사적인 방법이기 때문이므로..

In [74]:
# 대각 성분에 특이값을 가지는 대각 행렬 S를 대각행렬로 변환
S_diag = torch.diag(s)
restored_matrix = torch.matmul(torch.matmul(u, S_diag), v)
restored_matrix2 = u @ S_diag @ v

# random_matrix와 restored_matrix가 동일한지 확인
are_equal = torch.equal(random_matrix, restored_matrix)
print("random_matrix와 restored_matrix가 동일한가요?:", are_equal)

print("="*20)

print("원본 행렬:")
print(random_matrix)

print("복원 행렬:")
print(restored_matrix)

print("복원 행렬2 :")
print(restored_matrix2)

# 복원된 행렬과 원본 행렬 간의 차이 계산
difference = torch.abs(random_matrix - restored_matrix)
print("원본 행렬과 복원된 행렬 간의 차이:")
print(difference)

random_matrix와 restored_matrix가 동일한가요?: False
원본 행렬:
tensor([[0.0116, 0.5807, 0.2184],
        [0.0491, 0.7608, 0.3512],
        [0.2313, 0.3778, 0.8771]])
복원 행렬:
tensor([[ 0.2651,  0.0183, -0.5607],
        [ 0.3068, -0.0156, -0.7811],
        [-0.1566, -0.5050, -0.8282]])
복원 행렬2 :
tensor([[ 0.2651,  0.0183, -0.5607],
        [ 0.3068, -0.0156, -0.7811],
        [-0.1566, -0.5050, -0.8282]])
원본 행렬과 복원된 행렬 간의 차이:
tensor([[0.2535, 0.5623, 0.7791],
        [0.2577, 0.7764, 1.1323],
        [0.3880, 0.8828, 1.7053]])


## 2. 3차원 행렬에 대한 SVD

In [75]:
# 원본 행렬 생성 (3D 텐서, shape: (2, 3, 3))
original_matrix = torch.rand(2, 3, 3)
print("원본 행렬:")
print(original_matrix)

# 각 차원에 대해 SVD 수행
U, S, Vt = torch.svd(original_matrix)

print("특이값:")
print(S)

# 대각 성분에 특이값을 가지는 대각 행렬로 변환
S_diag = torch.zeros(2, 3, 3)  # 대각 행렬 초기화
S_diag[:, :3, :3] = torch.diag_embed(S)  # 대각 성분에 특이값 할당

print("특이값을 대각행렬로 변환:")
print(S_diag)

# 원본 행렬 복원
restored_matrix = torch.matmul(torch.matmul(U, S_diag), Vt)

print("복원된 원본 행렬:")
print(restored_matrix)

# 복원된 행렬과 원본 행렬 간의 차이 계산
difference = torch.abs(original_matrix - restored_matrix)
print("원본 행렬과 복원된 행렬 간의 차이:")
print(difference)

원본 행렬:
tensor([[[0.9919, 0.6000, 0.8292],
         [0.5598, 0.1551, 0.4423],
         [0.9618, 0.9073, 0.2264]],

        [[0.3039, 0.9535, 0.8147],
         [0.0834, 0.9186, 0.5493],
         [0.9926, 0.7731, 0.2885]]])
특이값:
tensor([[2.0163, 0.5363, 0.1049],
        [1.9768, 0.7410, 0.1652]])
특이값을 대각행렬로 변환:
tensor([[[2.0163, 0.0000, 0.0000],
         [0.0000, 0.5363, 0.0000],
         [0.0000, 0.0000, 0.1049]],

        [[1.9768, 0.0000, 0.0000],
         [0.0000, 0.7410, 0.0000],
         [0.0000, 0.0000, 0.1652]]])
복원된 원본 행렬:
tensor([[[ 0.8782, -0.0891,  1.1190],
         [ 0.4196, -0.2041,  0.5615],
         [ 1.1616,  0.2656,  0.6163]],

        [[ 0.7772, -1.0074,  0.2152],
         [ 0.6198, -0.8742, -0.0637],
         [-0.0019, -1.1345,  0.6157]]])
원본 행렬과 복원된 행렬 간의 차이:
tensor([[[0.1137, 0.6891, 0.2898],
         [0.1402, 0.3592, 0.1193],
         [0.1997, 0.6418, 0.3900]],

        [[0.4733, 1.9609, 0.5995],
         [0.5364, 1.7928, 0.6129],
         [0.9945, 1.9076, 0.3272]]]

## 2. 4차원 행렬에 대한 SVD

In [76]:
# 원본 텐서 생성 (4D 텐서, shape: (48, 48, 3, 3))
original_tensor = torch.rand(48, 48, 3, 3)
print("원본 텐서:")
print(original_tensor.shape)

# 각 차원에 대해 SVD 수행
U, S, Vt = torch.svd(original_tensor.view(-1, 3, 3))  # 텐서를 2D로 변환하여 SVD 수행

# 대각 성분에 특이값을 가지는 대각 행렬로 변환
S_diag = torch.diag_embed(S)  # 대각 성분에 특이값 할당

# 원본 텐서 복원 (4D로 변환)
restored_tensor = torch.matmul(torch.matmul(U, S_diag), Vt.transpose(1, 2)).view(48, 48, 3, 3)

print("복원된 원본 텐서:")
print(restored_tensor.shape)

# 복원된 텐서와 원본 텐서 간의 차이 계산
difference = torch.abs(original_tensor - restored_tensor)
print("원본 텐서와 복원된 텐서 간의 차이:")
print(difference.max())  # 차이의 최댓값 출력

원본 텐서:
torch.Size([48, 48, 3, 3])
복원된 원본 텐서:
torch.Size([48, 48, 3, 3])
원본 텐서와 복원된 텐서 간의 차이:
tensor(1.1325e-06)


In [77]:
print("특이값 shape : ")
print(S.shape)

print("특이값 S : ")
print(S)

print("특이값 대각성분의 shape : ")
print(S_diag.shape)

print("특이값 대각성분 : ")
print(S_diag)

특이값 shape : 
torch.Size([2304, 3])
특이값 S : 
tensor([[1.9178, 0.3308, 0.0274],
        [1.4175, 0.2445, 0.0413],
        [1.7326, 0.5388, 0.2170],
        ...,
        [1.2062, 0.7146, 0.2120],
        [1.6905, 0.4105, 0.0925],
        [1.2200, 0.4686, 0.2102]])
특이값 대각성분의 shape : 
torch.Size([2304, 3, 3])
특이값 대각성분 : 
tensor([[[1.9178, 0.0000, 0.0000],
         [0.0000, 0.3308, 0.0000],
         [0.0000, 0.0000, 0.0274]],

        [[1.4175, 0.0000, 0.0000],
         [0.0000, 0.2445, 0.0000],
         [0.0000, 0.0000, 0.0413]],

        [[1.7326, 0.0000, 0.0000],
         [0.0000, 0.5388, 0.0000],
         [0.0000, 0.0000, 0.2170]],

        ...,

        [[1.2062, 0.0000, 0.0000],
         [0.0000, 0.7146, 0.0000],
         [0.0000, 0.0000, 0.2120]],

        [[1.6905, 0.0000, 0.0000],
         [0.0000, 0.4105, 0.0000],
         [0.0000, 0.0000, 0.0925]],

        [[1.2200, 0.0000, 0.0000],
         [0.0000, 0.4686, 0.0000],
         [0.0000, 0.0000, 0.2102]]])


In [78]:
# 원본 텐서 생성 (5D 텐서, shape: (1. 48, 48, 3, 3))
original_tensor = original_tensor.unsqueeze(0)
u, s, v = torch.svd(original_tensor, some=False)  # SVD 수행

s_diag = torch.diag_embed(s)  # 대각 성분에 특이값 할당

In [79]:
print("특이값 shape : ")
print(s.shape)

print("특이값 s : ")
print(s)

print("특이값 대각성분 : ")
print(s_diag)

특이값 shape : 
torch.Size([1, 48, 48, 3])
특이값 s : 
tensor([[[[1.9178, 0.3308, 0.0274],
          [1.4175, 0.2445, 0.0413],
          [1.7326, 0.5388, 0.2170],
          ...,
          [1.7733, 0.5848, 0.0343],
          [1.4320, 0.4974, 0.0370],
          [1.7105, 0.7592, 0.0281]],

         [[1.8294, 0.7002, 0.1355],
          [1.7337, 0.5224, 0.0656],
          [1.3054, 0.7387, 0.3750],
          ...,
          [1.8165, 0.8181, 0.0862],
          [1.5817, 0.5849, 0.1458],
          [1.4178, 0.2443, 0.0280]],

         [[1.8651, 0.7963, 0.1113],
          [1.3457, 0.6369, 0.2333],
          [1.6249, 0.4140, 0.0324],
          ...,
          [1.7276, 0.6930, 0.4080],
          [1.6445, 0.3671, 0.0742],
          [1.7347, 0.2603, 0.0742]],

         ...,

         [[1.6395, 0.6481, 0.1512],
          [1.3839, 0.5675, 0.2783],
          [2.0733, 0.3838, 0.1866],
          ...,
          [1.4459, 0.7659, 0.0723],
          [1.2688, 0.2608, 0.1204],
          [1.9789, 0.6389, 0.3113]],

    